### Rastreo de noticias en Google News sobre temas de la Ciudad de México que competan o sean de interés del Instituto de Planeación de la Ciudad de México

El script extrae las notas de Google News de los temas de agua, movilidad, planeación, salud y seguridad de la Ciudad de México. El script extrae los títulos, url y fecha de las notas.

In [82]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import RGBColor
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Pt

Extraer notas a partir de los temas relevantes y palabras clave para localizarlos en Ciudad de México

In [83]:
# Temas para la búsqueda
temas = [
    "infraestructura", "agua", "pgd", "pgot", 
    "ordenamiento%20territorial", "movilidad", 
    "salud", "gestión%20de%20riesgos", "seguridad%20publica", 
    "participación%20ciudadana", "medio%20ambiente", "cultura", 
    "asentamientos%20irregulares", "ipdp", 
    "transporte%20publico", "planeacion", "espacio%20publico"
]

In [ ]:
#Palabras clave para filtrar solo noticias de la Ciudad de México
palabras_clave_cdmx = [
    "Ciudad de México", "CDMX", "Iztapalapa", "Coyoacán", "Cuauhtémoc", "Benito Juárez", 
    "Miguel Hidalgo", "Xochimilco", "Tlalpan", "Gustavo A. Madero", "Venustiano Carranza",
    "Azcapotzalco", "Tláhuac", "Álvaro Obregón", "Milpa Alta", "Magdalena Contreras"
]

In [ ]:
#Lista de medios relevantes
medios = [
    'infobae', 'La Prensa', 'Debate', 'MSN México', 'La Crónica de Hoy', 'El Universal',
    'REFORMA', 'MVS Noticias', 'SinEmbargo MX', 'Cuarto Poder', 'Reporte Indigo',
    'Eje Central', 'Gobierno de la Ciudad de México', 'La Silla Rota', 'MSN', 
    'Gobierno de México', 'Animal Político', 'Periódico AM', 'La Razón de México',
    'Periódico Excélsior', 'Radio Fórmula', 'N+', 'La Izquierda Diario', 'Milenio',
    'SSC-CdMx', 'Infobae México', 'TV Azteca', 'Quadratín México', 'ContraRéplica',
    'La Jornada', 'Uno TV Noticias', 'Secretaría de Gestión Integral de Riesgos y Protección Civil',
    'Telediario CDMX', 'Ovaciones', 'ADN 40', 'La-Lista', 'El Heraldo de México',
    'IECM', 'El Economista', 'RTVE', 'EL PAÍS'
]

#Convertir la lista de medios en una expresión regular
medios_regex = '|'.join(map(re.escape, medios))

In [ ]:
#Función para extraer notas
def get_notas(tema, periodo):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'}
    
    url = f'https://news.google.com/search?q={tema}%20ciudad%20de%20mexico%20when%3A{periodo}d&hl=es-419&gl=MX&ceid=MX%3Aes-419'
    response = requests.get(url, headers=headers)
    sopa = BeautifulSoup(response.text, 'html.parser')
    resultados = sopa.find_all('article')
    
    #Listas para almacenar datos
    titulos, urls, fechas, medios_lista = [], [], [], []

    for resultado in resultados:
        #Extraer título
        titulo_elem = resultado.find('a', class_='JtKRv')
        titulo = titulo_elem.text.strip() if titulo_elem else None

        #Extraer URL
        url_elem = resultado.find('a')
        url = 'https://news.google.com' + url_elem.get('href')[1:] if url_elem and url_elem.get('href') else None

        #Extraer fecha
        fecha_elem = resultado.find('time')
        fecha = fecha_elem['datetime'] if fecha_elem else None

        #Extraer medio
        medio_elem = resultado.find('div', class_='vr1PYe')
        medio = medio_elem.text.strip() if medio_elem else None

        #Guardar en listas
        titulos.append(titulo)
        urls.append(url)
        fechas.append(fecha)
        medios_lista.append(medio)

    #Crear DataFrame
    df = pd.DataFrame({
        'titulo': titulos,
        'url': urls,
        'fecha_nota': fechas,
        'medio': medios_lista,
        'fecha_consulta': pd.to_datetime('today').strftime('%Y-%m-%d'),
        'tema': tema
    })

    return df

In [ ]:
periodo =  1 #Definir el periodo de busqueda por días 

#Extraer notas de todos los temas en un dolo dataframe
consolidado_notas = pd.concat([get_notas(tema, periodo) for tema in temas], ignore_index=True)

#Filtrar notas de los medios definidos
consolidado_filtrado = consolidado_notas[consolidado_notas['medio'].fillna('').str.contains(medios_regex, regex=True)]

#Filtrar por palabras clave de CDMX
consolidado_final = consolidado_filtrado[consolidado_filtrado['titulo'].fillna('').str.contains('|'.join(palabras_clave_cdmx), regex=True, case=False)]


In [ ]:
#Mostrar resultados
print(f'Total de notas encontradas: {len(consolidado_notas)}')
print(f'Notas filtradas por medios: {len(consolidado_filtrado)}')
print(f'Notas finales filtrando por palabras clave: {len(consolidado_final)}')

Total de notas encontradas: 439
Notas filtradas por medios: 220
Notas finales filtrando por palabras clave: 108


In [ ]:
#Guardar en Excel
consolidado_final.to_excel("noticias_cdmx.xlsx", index=False)


Almacenar las notas en el formato de entrega

In [ ]:
#Función para agregar hipervínculos en las notas
def add_hyperlink(paragraph, text, url):
    """
    Agrega un hipervínculo a un párrafo en un documento de Word.
    """
    part = paragraph._parent.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)

    hyperlink = OxmlElement("w:hyperlink")
    hyperlink.set(qn("r:id"), r_id)

    run = OxmlElement("w:r")
    hyperlink.append(run)

    rPr = OxmlElement("w:rPr")
    color = OxmlElement("w:color")
    color.set(qn("w:val"), "0000FF")  #Azul
    rPr.append(color)

    underline = OxmlElement("w:u")
    underline.set(qn("w:val"), "single")
    rPr.append(underline)

    run.append(rPr)

    text_element = OxmlElement("w:t")
    text_element.text = text
    run.append(text_element)

    paragraph._element.append(hyperlink)

In [ ]:
#Cargar el archivo Excel
df = pd.read_excel("noticias_cdmx.xlsx")

In [ ]:
#Limpiar valores nulos en 'medio', 'url' y 'fecha_nota'
df['medio'] = df['medio'].fillna('Fuente desconocida')
df['url'] = df['url'].fillna('#')  # Evita errores en enlaces vacíos

#Formato año, mes y día eliminandio el formato de fecha de la busqueda
df['fecha_nota'] = pd.to_datetime(df['fecha_nota'], errors='coerce').dt.strftime('%Y-%m-%d')

#Reemplazar '%20' con espacios en los temas
df['tema'] = df['tema'].str.replace('%20', ' ', regex=False)

#Agrupar los temas por cada nota (para evitar duplicados de filas)
df_grouped = df.groupby(['fecha_nota', 'medio', 'titulo', 'url']).agg({'tema': lambda x: ', '.join(set(x))}).reset_index()

In [ ]:
#Crear un documento de Word
doc = Document()


In [ ]:
#Agregar el título y los subtítulos
doc.add_heading("Monitoreo de Medios y Redes Sociales", level=1)
doc.add_paragraph("Temas prioritarios de planeación de la Ciudad de México")
doc.add_paragraph(f"Informe con corte al día {pd.to_datetime("today").strftime('%d de %B de %Y')}")



In [ ]:
#Encabezados de la tabla
table = doc.add_table(rows=1, cols=4)
table.style = 'Table Grid'
hdr_cells = table.rows[0].cells
hdr_cells[0].text = "Fecha de la nota"
hdr_cells[1].text = "Fuente"
hdr_cells[2].text = "Idea central"
hdr_cells[3].text = "Temas"

In [103]:
#Llenar la tabla con los datos de las notas
for _, row in df_grouped.iterrows():
    row_cells = table.add_row().cells
    row_cells[0].text = row["fecha_nota"] if pd.notna(row["fecha_nota"]) else "No encontrado"
    
    #Agregar el hipervínculo en la celda del medio
    p = row_cells[1].paragraphs[0]
    add_hyperlink(p, row["medio"], row["url"])
    
    row_cells[2].text = row["titulo"]
    row_cells[3].text = row["tema"]


In [104]:
# Guardar el archivo en formato Word
doc.save("Monitoreo_Medios_y_Redes_.docx")
